In [18]:
import sys
sys.path.insert(1, '../src/')

import db_wrapper as db
import gemini_wrapper as gemini
import rag, re, json

In [5]:
gemini.configure()

In [2]:
doc, page = 0, 22
_, _, _, txt, emb = db.retrieve(doc, page)

In [4]:
query = 'What are the two programming methods used for QUANTEC?'
answer = 'The Quantec system was adapted to provide a bespoke solution for the RUH by the manufacturer.'
context = txt

In [9]:
template = open('../metadata/validation_template.md').read()
vjs = open('../metadata/valid_js_template.json').read()
prompt = template.format(query=query, answer=answer, context=context, valid_js_template=vjs)
open('../tmp/validation_prompt.md', 'w+').write(prompt)

1019

In [ ]:
def extract_json(txt):
    m = re.match('```json(.+)```', txt, re.DOTALL)
    if m is None:
        return txt
    
    return m.group(1).strip()    

def compose_validation(txt):
    js = json.loads(extract_json(txt))
    ans = 'The answer is valid.' if js['acceptable'] else 'The answer is not valid.'
    ans += '\n'
    ans += js['explanation']
    return ans


In [14]:
r = gemini.ask(prompt)
extract_json(r)

'{\n  "acceptable": false,\n  "explanation": "This answer was unacceptable because it does not address the question. The question asks about the programming methods used for Quantec, but the answer describes a system adaptation for a specific hospital and mentions bespoke hardware (call buttons and displays) rather than programming techniques.  The provided text offers no information about Quantec\'s programming methods."\n}'

In [16]:
print(extract_json(r))

{
  "acceptable": false,
  "explanation": "This answer was unacceptable because it does not address the question. The question asks about the programming methods used for Quantec, but the answer describes a system adaptation for a specific hospital and mentions bespoke hardware (call buttons and displays) rather than programming techniques.  The provided text offers no information about Quantec's programming methods."
}


In [23]:
valid = compose_validation(r)

report_template = open('../metadata/report_template.md').read()
report = report_template.format(query=query, answer=answer, context=context, valid=valid)
open('../tmp/report.md', 'w+').write(report)

1209